# Exercise: Titanic Database
    
The JSON file `titanic.json` contains information about 1000+ passengers that were aboard the Titanic when it sunk in 1912. It's a document database: not all entries have the same fields, since there is a lot of missing data.

In [24]:
!pip install pymongo


**1) Load the JSON file into a MongoDB database.**

In [25]:
import pymongo
from pymongo import MongoClient
import json

client = MongoClient('mongodb://localhost:27017/')
db = client['titanic']
collection = db['passengers']

with open('titanic.json', 'r') as file:
    titanic_data = json.load(file)


collection.insert_many(titanic_data)

print(f"Number of documents in the collection: {collection.count_documents({})}")


BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: titanic.passengers index: _id_ dup key: { _id: 1 }', 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 1}, 'op': {'_id': 1, 'survived': 0.0, 'name': 'Braund, Mr. Owen Harris', 'gender': 'male', 'age': 22.0, 'n_siblings_spouse': 1.0, 'n_parents_children': 0.0, 'ticket_number': 'A/5 21171', 'fare': 7.25, 'hometown': 'Bridgerule, Devon, England', 'boarded_from': 'Southampton', 'destination': "Qu'Appelle Valley, Saskatchewan, Canada", 'class': 3.0}}], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': []}

**2) Find out exactly how many passengers (number of documents) there are in the database.**

In [ ]:
passenger_count = collection.count_documents({})


print(f"There are {passenger_count} passengers recorded in the database.")

There are 1309 passengers recorded in the database.


**3) How old was passenger "Bourke, Mr. John"? (use the fields `name` and `age`)**

In [ ]:
# Search for "Bourke, Mr. John" in the database
bourke_mr_john = collection.find_one({'name': 'Bourke, Mr. John'})

print(f"John's age: {bourke_mr_john['age']}")



John's age: 40.0


**4) The field `survived` tells us whether a passenger survived (value 1) or not (value 0). Find out how many survived and how many did not (note: many have missing data).**

In [ ]:
# Counting survivors and non-survivors
survivors = collection.count_documents({"survived": 1})
non_survivors = collection.count_documents({"survived": 0})

print(f"Number of survivors: {survivors}")
print(f"Number of non-survivors: {non_survivors}")

Number of survivors: 342
Number of non-survivors: 549


**5) Who was the oldest survivor of the Titanic?**

In [ ]:
# Find the oldest survivor
oldest_survivor = collection.find_one({"survived": 1, "age": {"$ne": None}}, sort=[("age", pymongo.DESCENDING)])
print(f"The oldest survivor is {name} at {age} years old.")

The oldest survivor is Barkworth, Mr. Algernon Henry Wilson at 80.0 years old.


**6) Find the survival rate (survivors/total) for each ticket class. (use the field `class`. There were three: 1, 2, 3)**

In [ ]:
# Initialize a dictionary to hold the survival rates
survival_rates = {}

# Iterate over each class
for ticket_class in [1, 2, 3]:
    # Count total passengers in this class
    total_passengers = collection.count_documents({"class": ticket_class})
    # Count total survivors in this class
    survivors = collection.count_documents({"class": ticket_class, "survived": 1})
    
    # Calculate the survival rate for this class
    survival_rate = (survivors / total_passengers) if total_passengers > 0 else 0
    survival_rates[ticket_class] = survival_rate

# Output the survival rates for each class
for ticket_class, rate in survival_rates.items():
    print(f"Survival rate for class {ticket_class}: {rate:.2%}")


Survival rate for class 1: 42.02%
Survival rate for class 2: 31.62%
Survival rate for class 3: 16.71%


**7) Which five passengers paid the five highest ticket prices? (use the field `fare`)**

In [ ]:
# Find the top five highest fares
top_fares = collection.find({}, {"name": 1, "fare": 1}).sort("fare", pymongo.DESCENDING).limit(5)

for passenger in top_fares:
    print(f"{passenger['name']} paid a fare of {passenger['fare']}")


Lesurer, Mr. Gustave J paid a fare of 512.3292
Ward, Miss. Anna paid a fare of 512.3292
Cardeza, Mrs. James Warburton Martinez (Charlotte Wardle Drake) paid a fare of 512.3292
Cardeza, Mr. Thomas Drake Martinez paid a fare of 512.3292
Fortune, Miss. Mabel Helen paid a fare of 263.0
